In [ ]:
import pandas as pd


# First, we need to remove all of the unesseary features, since i picked all beach breaks on the pacific west coast, we are treating
# this as one surf break, no need to include directios relative to the shore line nor location/date/times.

# Load the dataset
file_path = 'waves_fileV4b.csv'
data = pd.read_csv(file_path)

# Define secondary swell columns to check
secondary_swell_cols = [
    "Seconday 1 -> height", "Seconday 1 -> period", "Seconday 1 -> letters", "Seconday 1 -> angle", "Seconday 1
    "Seconday 2 -> height", "Seconday 2 -> period", "Seconday 2 -> letters", "Seconday 2 -> angle",
    "Seconday 3 -> height", "Seconday 3 -> period", "Seconday 3 -> letters", "Seconday 3 -> angle"
]

# Iterate through rows and check for missing secondary swell values
for index, row in data.iterrows():
    for col in secondary_swell_cols:
        if pd.isna(row[col]) or row[col] == '':
            data.at[index, col] = None  # Replace missing or empty values with None

# Dropping Columns that are not nessesary for predicitng surf quality

# List of columns to drop
columns_to_drop = [
    "DocumentID", "DatesDocumentID", "timestamp", "name", "County", "Name", "data",
    "Surf Height: Direction", "Surf Height: Angle", 
    "Seconday 1 -> letters", "Seconday 1 -> angle", 
    "Primary -> letters", "Primary -> angle", 
    "Seconday 2 -> letters", "Seconday 2 -> angle", 
    "Seconday 3 -> letters", "Seconday 3 -> angle", 
    "Wind Direction (NSEW)", "Wind Direction (Angle)"
]

# Drop the columns from the DataFrame
data_swell = data.drop(columns=columns_to_drop)

# shuffling all lines, forecasts are cumlative which means it would be biased if we split the data without shuffling
data_swell = data_swell.sample(frac=1, random_state=42).reset_index(drop=True)



In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


data = data_swell

# Separate features and target
X = data.drop(columns=['Star Rating'])  # Features
y = data['Star Rating']                # Target

X = pd.get_dummies(X, columns=["Wind Direction (Human Relation)"], drop_first=True)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Input layer
    Dense(32, activation='relu'),                                          # Hidden layer
    Dense(1)                                                               # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=50,       # Adjust as needed
    batch_size=32    # Adjust as needed
)

2024-11-30 14:39:45.362186: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'X_train_scaled' is not defined